In [1]:
import os, sys
import django
sys.path.append('../') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
ultrasounds = UltraSound.objects.all()

In [3]:
for ultrasound in ultrasounds:
    try:
        if (ultrasound.ga_confirmed > 16 and ultrasound.ga_confirmed < 30) and (ultrasound.ga_confirmed_after < 16 or ultrasound.ga_confirmed_after > 30):
            print(ultrasound.child_subject_identifier)
    except forms.ValidationError:
        subject_identifier = ultrasound.subject_identifier
        childconsents = CaregiverChildConsent.objects.filter(
            subject_consent__subject_identifier=subject_identifier)
        for consent in childconsents:
            cohort = Cohort.objects.filter(
                subject_identifier=consent.subject_identifier,
                current_cohort=True).latest('assign_datetime')
            if cohort.name == 'cohort_a':
                onschedule_model = ultrasound.maternal_visit.schedule.onschedule_model
                onschedule_model_cls = django_apps.get_model(onschedule_model)
                try:
                    onschedule_obj = onschedule_model_cls.objects.get(
                        subject_identifier=subject_identifier,
                        schedule_name=ultrasound.maternal_visit.schedule_name)
                except onschedule_model_cls.DoesNotExist:
                    print('does not exist', subject_identifier)
                else:
                    if onschedule_obj.child_subject_identifier == subject_identifier:
                        onschedule_obj.child_subject_identifier = consent.subject_identifier
                        onschedule_obj.save()
                    if ultrasound.child_subject_identifier == subject_identifier:
                        ultrasound.child_subject_identifier = consent.subject_identifier
                        ultrasound.save_base(raw=True)
                    print('childidx', onschedule_obj.child_subject_identifier)
        print('error', ultrasound.child_subject_identifier)


B142-040990625-1-10
B142-040991122-8-10
B142-040991190-5-10
B142-040990919-8-10
